In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt

In [2]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e' or base == 'Delta':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

In [3]:
simp = True

## Target function

we want to minimize the effective price, given a set of exponents for the buying and selling asset each.

In [4]:
def deltaBySpot_(asset, s,v, b, w, j, e, Delta, a, min):
  f = (s - (v + b) * w) / w
  if asset == 'b':
    f = -f
  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(asset, *all_symbols[asset]) for asset in assets}
deltaBySpot['b']

b_b - s_b/w_b + v_b

In [5]:
deltaBySpot['s']

-b_s + s_s/w_s - v_s

In [6]:
def spotByExp_(s, v, b, w, j, e, Delta, a, min):
  f = a * ((1 + 1/j) ** e)
  if simp:
    f = simplify(f)
  return f
spotByExp = {asset: spotByExp_(*all_symbols[asset]) for asset in assets}
spotByExp['b']

a_b*((j_b + 1)/j_b)**e_b

In [7]:
def deltaByExp_(asset, s, v, b, w, j, e, Delta, a, min):
    f = deltaBySpot[asset].subs(s, spotByExp[asset])
    if simp:
        f = simplify(f)
    return f

deltaByExp = {asset: deltaByExp_(asset, *all_symbols[asset]) for asset in assets}
deltaByExp['b']

-a_b*j_b**(-e_b)*(j_b + 1)**e_b/w_b + b_b + v_b

In [8]:
eff = Delta_s / Delta_b
eff

Delta_s/Delta_b

In [9]:
effByExps = eff.subs({Delta_b: deltaByExp['b'], Delta_s: deltaByExp['s']})
if simp:
    effByExps = simplify(effByExps)
effByExps

-j_b**e_b*j_s**(-e_s)*w_b*(a_s*(j_s + 1)**e_s - j_s**e_s*w_s*(b_s + v_s))/(w_s*(a_b*(j_b + 1)**e_b - j_b**e_b*w_b*(b_b + v_b)))

### convexity of the target function

- deltaBySpot is linear
- spotByExp is convex
- deltaByExp is a composition of the two

=> deltaByExp is convex

- negation flips convexity/concavity
- deltaBuyingByExp is negative => -deltaBuyingByExp is positive
- inversion of a positive function flips convexity/concavity => 1 / (-deltaBuyingByExp) is convex
- multiplying two nonnegative functions with different variables, that are each convex and independent of the other's variable, results in a convex function

==> effectivePrice = deltaSellingByExp / (-deltaBuyingByExp) is convex. qed


## inequality-constraints

- value in A0 of buying must not exceed that of selling
- the exponents must adhere to their upper (buying) resp. lower (selling) bounds given by our equation
- need to buy and sell minimum amounts (TODO)
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not exceed expLimit (TODO)

In [10]:
a0Buying = spotByExp['s'] * deltaByExp['b']
if simp:
    a0Buying = simplify(a0Buying)
a0Buying

a_s*j_b**(-e_b)*((j_s + 1)/j_s)**e_s*(-a_b*(j_b + 1)**e_b + j_b**e_b*w_b*(b_b + v_b))/w_b

In [11]:
a0Selling = spotByExp['b'] * deltaByExp['s']
if simp:
    a0Selling = simplify(a0Selling)
a0Selling

a_b*j_s**(-e_s)*((j_b + 1)/j_b)**e_b*(a_s*(j_s + 1)**e_s - j_s**e_s*w_s*(b_s + v_s))/w_s

In [12]:
a0Bound = a0Buying - a0Selling # <=! 0
if simp:
    a0Bound = simplify(a0Bound)
a0Bound

-j_b**(-2*e_b)*j_s**(-2*e_s)*(a_b*j_s**e_s*w_b*(j_b*(j_b + 1))**e_b*(a_s*(j_s + 1)**e_s - j_s**e_s*w_s*(b_s + v_s)) + a_s*j_b**e_b*w_s*(j_s*(j_s + 1))**e_s*(a_b*(j_b + 1)**e_b - j_b**e_b*w_b*(b_b + v_b)))/(w_b*w_s)

In [13]:
def expBound_(asset, s, v, b, w, j, e, Delta, a, min):
    e_bound = log(w * (v + b) /a, 1 + 1/j) # constant wrt e
    if simp:
        e_bound = simplify(e_bound)
    if asset == 'b':
        f = e - e_bound # <=! 0 (upper bound)
    else:
        f = e_bound - e # <=! 0 (lower bound)
    if simp:
        f = simplify(f)
    return f

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

e_b - log(a_b**(-1/log((j_b + 1)/j_b))*(w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b)))

In [14]:
expBound['s']

-e_s + log(a_s**(-1/log((j_s + 1)/j_s))*(w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s)))

In [16]:
max_s, I_max = symbols('max_s I_max', positive=True, integer=True)

def maxSpotBound_(asset, s, v, b, w, j, e, Delta, a, min):
  return spotByExp[asset] - I_max # <=! 0

maxSpotBound = {asset: maxSpotBound_(asset, *all_symbols[asset]) for asset in assets}
maxSpotBound['b']

-I_max + a_b*((j_b + 1)/j_b)**e_b

In [17]:
def minAmntBound_(asset, s, v, b, w, j, e, Delta, a, min):
    f = min - Delta # <=! 0
    f = f.subs(Delta, deltaByExp[asset])
    if simp:
        f = simplify(f)
    return f

minAmntBound = {asset: minAmntBound_(asset, *all_symbols[asset]) for asset in assets}
minAmntBound['b']

a_b*j_b**(-e_b)*(j_b + 1)**e_b/w_b - b_b + min_b - v_b

In [18]:
minAmntBound['s']

-a_s*j_s**(-e_s)*(j_s + 1)**e_s/w_s + b_s + min_s + v_s

In [19]:
def maxAmntBound_(asset, s, v, b, w, j, e, Delta, a, min):
    if asset == 'b':
        max = b
    else:
        max = max_s
    f = Delta - max # <=! 0
    f = f.subs(Delta, deltaByExp[asset])
    if simp:
        f = simplify(f)
    return f

maxAmntBound = {asset: maxAmntBound_(asset, *all_symbols[asset]) for asset in assets}
maxAmntBound['b']

-a_b*j_b**(-e_b)*(j_b + 1)**e_b/w_b + v_b

In [20]:
maxAmntBound['s']

a_s*j_s**(-e_s)*(j_s + 1)**e_s/w_s - b_s - max_s - v_s

## Lagrangian

In [21]:
l1, l2, l3, l4, l5, l6, l7, l8, l9 = symbols('lambda_1 lambda_2 lambda_3 lambda_4 lambda_5 lambda_6 lambda_7 lambda_8 lambda_9', positive=True)

L = effByExps + + l1 * a0Bound + l2 * expBound['b'] + l3 * expBound['s'] + l4 * maxSpotBound['b'] + l5 * maxSpotBound['s'] + l6 * minAmntBound['b'] + l7 * minAmntBound['s'] + l8 * maxAmntBound['b'] + l9 * maxAmntBound['s']
if simp:
    L = simplify(L)
L

j_b**(-2*e_b)*j_s**(-2*e_s)*(j_b**(3*e_b)*j_s**e_s*w_b**2*(-a_s*(j_s + 1)**e_s + j_s**e_s*w_s*(b_s + v_s)) + j_b**(2*e_b)*j_s**(2*e_s)*w_b*w_s*(a_b*(j_b + 1)**e_b - j_b**e_b*w_b*(b_b + v_b))*(lambda_2*(e_b - log(a_b**(-1/log((j_b + 1)/j_b))*(w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b)))) - lambda_3*(e_s - log(a_s**(-1/log((j_s + 1)/j_s))*(w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s))))) - j_b**(2*e_b)*j_s**e_s*lambda_5*w_b*w_s*(I_max*j_s**e_s - a_s*(j_s + 1)**e_s)*(a_b*(j_b + 1)**e_b - j_b**e_b*w_b*(b_b + v_b)) + j_b**(2*e_b)*j_s**e_s*w_b*(a_b*(j_b + 1)**e_b - j_b**e_b*w_b*(b_b + v_b))*(lambda_7*(-a_s*(j_s + 1)**e_s + j_s**e_s*w_s*(b_s + min_s + v_s)) - lambda_9*(-a_s*(j_s + 1)**e_s + j_s**e_s*w_s*(b_s + max_s + v_s))) - j_b**e_b*j_s**(2*e_s)*lambda_4*w_b*w_s*(I_max*j_b**e_b - a_b*(j_b + 1)**e_b)*(a_b*(j_b + 1)**e_b - j_b**e_b*w_b*(b_b + v_b)) + j_b**e_b*j_s**(2*e_s)*w_s*(a_b*(j_b + 1)**e_b - j_b**e_b*w_b*(b_b + v_b))*(lambda_6*(a_b*(j_b + 1)**e_b + j_b**e_b*w_b*(-b_b + min_b - v_b)) - lambda_8

In [22]:
# Partial derivatives
dL_des = L.diff(e_s)
dL_deb = L.diff(e_b)

# Set them to zero
stationary_conditions = [
    Eq(dL_des, 0),
    Eq(dL_deb, 0),
]

In [23]:
solutions_ = solve(stationary_conditions, (e_s, e_b, l1, l2, l3, l4, l5, l6, l7, l8, l9))

print(len(solutions_))
solutions = solutions_[0]
e_s_sol, e_b_sol, l1_sol, l2_sol, l3_sol, l4_sol, l5_sol, l6_sol, l7_sol, l8_sol, l9_sol = solutions
solutions

In [ ]:
for sol in solutions:
  print(sol.free_symbols)

In [ ]:
e_s_sol

In [ ]:
e_b_sol

In [ ]:
l1_sol

In [ ]:
l2_sol

In [ ]:
l3_sol

In [ ]:
l4_sol

In [ ]:
l5_sol

In [ ]:
l6_sol

In [ ]:
l7_sol

In [ ]:
l8_sol

In [ ]:
l9_sol